In [3]:
# Load transformed_full .csv and transformed_incremental.csv files from user/ET_Exam_Allan_095 directory into sqlite3 database file
import sqlite3
import pandas as pd
import os

# Define file paths
full_csv = r"C:\Users\user\ET_Exam_Allan_095\data\transformed\transformed_full.csv"
incremental_csv = r"C:\Users\user\ET_Exam_Allan_095\data\transformed\transformed_incremental.csv"
db_path = r"C:\Users\user\ET_Exam_Allan_095\data\transformed\full_data.db"

# Ensure the directory exists
os.makedirs(os.path.dirname(db_path), exist_ok=True)

# Connect to SQLite database (creates file if it doesn't exist)
conn = sqlite3.connect(db_path)

# Load CSVs into DataFrames
df_full = pd.read_csv(full_csv)
df_incremental = pd.read_csv(incremental_csv)

# Write to SQLite tables
df_full.to_sql("transformed_full", conn, if_exists="replace", index=False)
df_incremental.to_sql("transformed_incremental", conn, if_exists="replace", index=False)

# Close connection
conn.close()

print(f"Data loaded successfully into {db_path}")

C:\Users\user\AppData\Local\Temp\ipykernel_5172\1864951124.py:18: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv(full_csv)


Data loaded successfully into C:\Users\user\ET_Exam_Allan_095\data\transformed\full_data.db


In [8]:
# Verifying the load by running the sql query
db_path = r"C:\Users\user\ET_Exam_Allan_095\data\transformed\full_data.db"

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Helper function to run and print query results
def run_query(description, query):
    print(f"\n {description}")
    print("-" * 50)
    cursor.execute(query)
    rows = cursor.fetchall()
    if rows:
        # Get column names
        col_names = [description[0] for description in cursor.description]
        print(" | ".join(col_names))
        print("-" * 50)
        for row in rows:
            print(" | ".join(str(value) for value in row))
    else:
        print("No results returned.")

# 1. Check table names
print(" Checking if expected tables exist...")
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [row[0] for row in cursor.fetchall()]
print("Tables in database:", tables)

if "transformed_full" not in tables or "transformed_incremental" not in tables:
    print("  Warning: One or both tables are missing!")
else:
    # 2. Row counts
    run_query("Row count in transformed_full", "SELECT COUNT(*) AS row_count FROM transformed_full;")
    run_query("Row count in transformed_incremental", "SELECT COUNT(*) AS row_count FROM transformed_incremental;")

# Close connection
conn.close()
print("\n Verification complete.")

 Checking if expected tables exist...
Tables in database: ['transformed_full', 'transformed_incremental']

 Row count in transformed_full
--------------------------------------------------
row_count
--------------------------------------------------
526052

 Row count in transformed_incremental
--------------------------------------------------
row_count
--------------------------------------------------
87356

 Verification complete.


In [ ]:
# Previewing the output
db_path = r"C:\Users\user\ET_Exam_Allan_095\data\transformed\full_data.db"

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

def inspect_table(table_name, sample_size=5):
    print(f"\n{'='*60}")
    print(f" INSPECTING TABLE: {table_name}")
    print('='*60)

    # 1. Row count
    cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
    count = cursor.fetchone()[0]
    print(f" Record count: {count}")

    # 2. Column names and inferred types
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    col_names = [col[1] for col in columns]
    col_types = [col[2] for col in columns]

    print(f" Column names ({len(col_names)}):")
    for i, (name, dtype) in enumerate(zip(col_names, col_types), 1):
        print(f"   {i}. {name} → {dtype or 'TEXT (inferred)'}")

    # 3. Sample data (5–10 rows)
    cursor.execute(f"SELECT * FROM {table_name} LIMIT {sample_size};")
    rows = cursor.fetchall()

    if rows:
        print(f"\n Sample rows (first {min(sample_size, len(rows))}):")
        # Print header
        print(" | ".join(f"{name:15}" for name in col_names))
        print("-" * (17 * len(col_names)))
        # Print rows
        for row in rows:
            print(" | ".join(f"{str(val):15}" for val in row))
    else:
        print(" No data found in table.")

# Inspect both tables
inspect_table("transformed_full", sample_size=5)
inspect_table("transformed_incremental", sample_size=5)

# Final summary
print(f"\n{'='*60}")
print(" VERIFICATION COMPLETE")
print('='*60)

conn.close()


 INSPECTING TABLE: transformed_full
 Record count: 526052
 Column names (10):
   1. InvoiceNo → TEXT
   2. StockCode → TEXT
   3. Description → TEXT
   4. Quantity → INTEGER
   5. InvoiceDate → TEXT
   6. UnitPrice → REAL
   7. CustomerID → INTEGER
   8. Country → TEXT
   9. InvoiceDate_iso → TEXT
   10. TotalPrice → REAL

 Sample rows (first 5):
InvoiceNo       | StockCode       | Description     | Quantity        | InvoiceDate     | UnitPrice       | CustomerID      | Country         | InvoiceDate_iso | TotalPrice     
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
536365          | 85123A          | WHITE HANGING HEART T-LIGHT HOLDER | 6               | 2010-12-01 08:26:00 | 2.55            | 17850           | United Kingdom  | 2010-12-01 08:26:00 | 15.3           
536365          | 71053           | WHITE METAL LANTERN | 6               | 2010-12-01 08:26:00 

In [9]:
# Saving the outputs into loaded folder in user/ET_Exam_Allan_095 directory
full_csv = r"C:\Users\user\ET_Exam_Allan_095\data\transformed\transformed_full.csv"
incremental_csv = r"C:\Users\user\ET_Exam_Allan_095\data\transformed\transformed_incremental.csv"

# Output database path → inside 'loaded' folder
output_dir = r"C:\Users\user\ET_Exam_Allan_095\data\loaded"
os.makedirs(output_dir, exist_ok=True)  # Create folder if it doesn't exist
db_path = os.path.join(output_dir, "full_data.db")

# Connect to SQLite (creates file in 'loaded/' folder)
conn = sqlite3.connect(db_path)

# Load and write CSVs
df_full = pd.read_csv(full_csv)
df_incremental = pd.read_csv(incremental_csv)

df_full.to_sql("transformed_full", conn, if_exists="replace", index=False)
df_incremental.to_sql("transformed_incremental", conn, if_exists="replace", index=False)

conn.close()

print(f" Database saved to: {db_path}")

C:\Users\user\AppData\Local\Temp\ipykernel_5172\214273912.py:14: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv(full_csv)


 Database saved to: C:\Users\user\ET_Exam_Allan_095\data\loaded\full_data.db
